In [1]:
!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 492.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.0+cu121
    Uninstalling torchvision-0.18.0+cu121:
      Successfully uninstalled torchvision-0.18.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.3.0+cu121
    Uninstalling tor

In [2]:
!pip install kaolin==0.15.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.1.1_cu121.html

Looking in links: https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.1.1_cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.7 MB/s eta 0:00:00
  Created wheel for pygltflib: filename=pygltflib-1.16.2-py3-none-any.whl size=27300 sha256=a6f9108ebb89119ee9b97836358a742884cdfa3412d9c55a0314546bce7e8574
  Stored in directory: /root/.cache/pip/wheels/4a/95/a2/2ee16c602170b0601ebeddbf06597868f09b7dbb0495ae99c4
Successfully built pygltflib


In [3]:
import kaolin
print(kaolin.__version__)

0.15.0


In [4]:
#improter

import random
import os
import re
import sys
import json
import time
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm

In [12]:
#connect to GoogleDrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
#dataset_download

def extract(filename):
    #zip_path = f'/content/other_drive/MyDrive/dataset_2d_to_point_clouds/{filename}'
    zip_path = f'/content/drive/MyDrive/dataset_2d_to_point_clouds/{filename}'
    extract_path = '/content/extracted_files'
    os.makedirs(extract_path, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    extracted_files = os.listdir(extract_path)
    print(extracted_files)

#print(os.listdir('/content/extracted_files/ShapeNet_pointclouds/02691156/1a04e3eab45ca15dd86060f189eb133'))

In [14]:
#unzipping dataset

extract('ShapeNetRendering.zip')
extract('ShapeNet_pointclouds.zip')
extract('splits.zip')

['ShapeNetRendering']
['ShapeNet_pointclouds', 'ShapeNetRendering']
['ShapeNet_pointclouds', 'ShapeNetRendering', 'splits']


In [15]:
print(os.listdir('/content/extracted_files/ShapeNet_pointclouds/02691156/1a04e3eab45ca15dd86060f189eb133'))

['pointcloud_2048.npy', 'pointcloud_1024.npy']


In [16]:
#BatchGen for DenceNet, (point cloud(1024 points) --- point cloud(2048 points))

shapenet_id_to_category = {
'02691156': 'airplane',
'02828884': 'bench',
'02933112': 'cabinet',
'02958343': 'car',
'03001627': 'chair',
'03211117': 'monitor',
'03636649': 'lamp',
'03691459': 'speaker',
'04090263': 'rifle',
'04256520': 'sofa',
'04379243': 'table',
'04401088': 'telephone',
'04530566': 'vessel'
}

shapenet_category_to_id = {
'airplane'	: '02691156',
'bench'		: '02828884',
'cabinet'	: '02933112',
'car'		: '02958343',
'chair'		: '03001627',
'lamp'		: '03636649',
'monitor'	: '03211117',
'rifle'		: '04090263',
'sofa'		: '04256520',
'speaker'	: '03691459',
'table'		: '04379243',
'telephone'	: '04401088',
'vessel'	: '04530566'
}

class BatchGen_DenceNet:

    def __init__(self, json_path, dots_directory):
        self.labels = []
        cats = shapenet_id_to_category.keys()

        with open(json_path, 'r') as f:
            train_models_dict = json.load(f)

        for cat in cats:
            self.labels.extend([model for model in train_models_dict[cat]])

        self.dots_directory = dots_directory

    def __len__(self):
        return len(self.labels)

    def transform_dots(self, dots):
        shape = dots.shape[0]

        if shape <= self.dots_size:

            for i in range(self.dots_size - shape):
                dots = pd.concat([dots, pd.DataFrame({'x': [0], 'y': [0], 'z':[0]})])
        else:
            indexes = np.random.choice(shape, shape - self.dots_size, replace=False)
            dots = dots.drop(indexes)

        return dots

    def normilize_dots(self, points):
        points -= np.min(points)
        points /= np.max(points)
        return points

    def __getitem__(self, index):
        label = self.labels[index]
        dots_path1 = os.path.join(self.dots_directory, os.path.join(label, 'pointcloud_1024.npy'))
        dots_path2 = os.path.join(self.dots_directory, os.path.join(label, 'pointcloud_2048.npy'))

        dots1 = np.load(dots_path1)
        dots2 = np.load(dots_path2)

        dots1 = self.normilize_dots(dots1)
        dots2 = self.normilize_dots(dots2)

        dots1 = torch.from_numpy(dots1).float()
        dots2 = torch.from_numpy(dots2).float()
        return dots1, dots2

In [17]:
# func for extracting local features

# This function finds 'num_near_points' nearest points for a given point. This will be used for extracting local point features.
# If there are less then 'num_near_points' points inside a circle of radius=radius, returned ndarray will be filled with [0, 0, 0];
# it will contain [0, 0, 0] * (num_near_points - num of points inside circle) times.

def get_nearest_points(pcl, radius=0.5, num_near_points=10):
    points = None
    for step, pcl_i in enumerate(pcl):
        nbrs = NearestNeighbors(n_neighbors=num_near_points, radius=radius, algorithm='auto').fit(pcl_i)
        distances, indices = nbrs.radius_neighbors(pcl_i)
        n = pcl_i.shape[0]
        nearest_points_all = np.zeros((n, num_near_points, 3))
        for i in range(n):
            nearest_points_within_radius = pcl_i[indices[i]]
            if len(nearest_points_within_radius) >= num_near_points:
                sorted_indices = np.argsort(distances[i])[:num_near_points]
                nearest_points_all[i] = nearest_points_within_radius[sorted_indices]
            else:
                nearest_points_all[i, :len(nearest_points_within_radius)] = nearest_points_within_radius

        nearest_points_all = np.expand_dims(nearest_points_all, axis=0)

        if step == 0:
            points = nearest_points_all
        else:

            points = np.concatenate((points, nearest_points_all), axis=0)

    return points

def concatenate_with_pcl(pcl, nearest_points_all):
    nearest_points_all_torch = torch.tensor(nearest_points_all, dtype=torch.float32).to('cuda')
    nearest_points_all_flattened = nearest_points_all_torch.view(nearest_points_all_torch.shape[0], nearest_points_all_torch.shape[1], -1)
    result = torch.cat((pcl, nearest_points_all_flattened), dim=2)
    return result

In [18]:
#DenceNet

class RadiusEncoder(nn.Module):
    def __init__(self, input_size, n_filters=[32, 128, 512, 2048, 512, 128, 64]): #n_filters[-1], (n/2**n_filters >= 1)
        super().__init__()
        n_layers = len(n_filters)
        self.dropouts = nn.ModuleList([nn.Dropout(0.3) for i in range(n_layers)])
        self.encoder = nn.ModuleList(
            [nn.Conv1d(input_size, n_filters[i], 3, padding=1) if i == 0 else nn.Conv1d(n_filters[i - 1], n_filters[i], 3, padding=1) #k, k-1//2  padding = (kernel_size-1)/2
             for i in range(n_layers)])
        self.pools = nn.ModuleList([nn.MaxPool1d(2) for i in range(n_layers)])

    def forward(self, x):
        n = x.shape[2]
        for conv, drop in zip(self.encoder, self.dropouts):
            x = conv(x)
            x = drop(x)
            x = F.relu(x)
        return x


class Encoder(nn.Module):
    def __init__(self, input_size, n_filters=[32, 128, 512, 2048, 512, 128, 64]): #n_filters[-1], (n/2**n_filters >= 1)
        super().__init__()
        n_layers = len(n_filters)
        self.dropouts = nn.ModuleList([nn.Dropout(0.3) for i in range(n_layers)])
        self.encoder = nn.ModuleList(
            [nn.Conv1d(input_size, n_filters[i], 3, padding=1) if i == 0 else nn.Conv1d(n_filters[i - 1], n_filters[i], 3, padding=1) #k, k-1//2  padding = (kernel_size-1)/2
             for i in range(n_layers)])
        self.pools = nn.ModuleList([nn.MaxPool1d(2) for i in range(n_layers)])

    def forward(self, x):
        n = x.shape[2]
        for conv, pool, drop in zip(self.encoder, self.pools, self.dropouts):
            x = conv(x)
            x = drop(x)
            x = pool(x)
            x = F.relu(x)
        x = torch.mean(x, dim=2)
        return x


class Decoder(nn.Module):
    def __init__(self, input_size, layer_sizes=[512, 1024, 2048, 1024, 512, 3]):
        super().__init__()
        n_layers = len(layer_sizes)
        self.decoder = nn.ModuleList()
        for i in range(n_layers - 1):
            if i == 0:
                self.decoder.append(nn.Conv1d(input_size, layer_sizes[i], 2, stride=2))
            else:
                self.decoder.append(nn.Conv1d(layer_sizes[i - 1], layer_sizes[i], 2, stride=2))

            self.decoder.append(nn.ConvTranspose1d(layer_sizes[i], layer_sizes[i], 2, stride=2))
            self.decoder.append(nn.ReLU())

        self.out = nn.Conv1d(layer_sizes[-2], layer_sizes[-1], 1)

    def forward(self, x):
        for layer in self.decoder:
            x = layer(x)
        x = self.out(x)
        return x


class DenseNet(nn.Module):
    def __init__(self, input_size, upsampling_factor): #upsampling_factor=2, num_radius_points=10
        super(DenseNet, self).__init__()
        self.upsampling_factor = upsampling_factor
        self.encoder = Encoder(input_size)
        self.radius_encoder = RadiusEncoder(input_size + 3 * 10)
        self.decoder = Decoder(131 + 128)

    def forward(self, pcl):
        batch_size = pcl.shape[0]
        n = pcl.shape[1]

        z = self.encoder(pcl.permute(0, 2, 1))
        #print('z: ', z.shape)
        #print('pcl_in: ', pcl.shape)

        point_feat = pcl.unsqueeze(2).expand(-1, -1, self.upsampling_factor, -1)  # (bs,NUM_POINTS,3) --> (bs,NUM_POINTS,1,3) --> (bs,NUM_POINTS,2,3)
        point_feat = point_feat.reshape(batch_size, -1, 3)  # (bs,NUM_POINTS,2,3) --> (bs,NUM_UPSAMPLE_POINTS,3)
        #print('point_feat: ', point_feat.shape)  # 'point_feat: ', point_feat.shape

        global_feat = z.unsqueeze(1).expand(-1, self.upsampling_factor*n, -1)  # (bs,bneck) --> (bs,1,bneck) --> (bs,NUM_UPSAMPLE_POINTS,bneck)
        #print('global_feat: ', global_feat.shape)  # 'global_feat: ', global_feat.shape
        concat_feat = torch.cat([point_feat, global_feat], dim=2).permute(0, 2, 1)
        #print('concat_feat1: ', concat_feat.shape)
        # adding local point features (3 circles)

        for i in range(3):
            near_points = get_nearest_points(pcl.cpu(), radius=0.03 + i/100, num_near_points=10)
            data = concatenate_with_pcl(pcl, near_points)
            data = data.transpose(1, 2)
            local_feat = self.radius_encoder(data)
            local_feat = local_feat.unsqueeze(2).repeat(1, 1, self.upsampling_factor, 1)  # (bs, NUM_POINTS, 1, 64) --> (bs, NUM_POINTS, 4, 64)
            local_feat = local_feat.view(batch_size, self.upsampling_factor*n, -1)  # (bs, NUM_POINTS, 4, 64) --> (bs, NUM_UPSAMPLE_POINTS, 64)
            #print("concat_feat", concat_feat.shape)
            #print("local_feat", local_feat.shape)
            local_feat = torch.transpose(local_feat, 1, 2)

            concat_feat = torch.cat([concat_feat, local_feat], dim=1)

        #print('concat_feat2: ', concat_feat.shape)
        dense = self.decoder(concat_feat)
        #print('dense1: ', dense.shape)
        dense = dense.view(batch_size, -1, 3)  # Reshape to (bs,NUM_UPSAMPLE_POINTS,3)
        #print('dense2: ', dense.shape)

        return dense


    def predict(self, x, upsample=2):
        for _ in range(upsample):
            x = self.forward(x)
        return x # [batch_size, N, 3] -> [batch_size, N * 2^upsample, 3]

In [ ]:
#DenseNet train

def point_loss(points):
    loss = torch.empty(points.shape[0], dtype=torch.float)
    loss = loss.to('cuda')
    for i in range(points.shape[0]):
        dist = torch.cdist(points[0], points[0])
        dist = dist[(dist <= 0.01) & (dist >= 0.00001)]
        dist = 0.002 / dist
        loss[i] = dist.mean()
    loss = min(1.0, loss.mean() / 5.0)
    return loss

batch_size = 32
upsampling_factor = 2
input_size = 3

pcl_path = '/content/extracted_files/ShapeNet_pointclouds'
json_path = '/content/extracted_files/splits/train_models.json'
params = {'batch_size': 32, 'shuffle': True}

training_set = BatchGen_DenceNet(json_path, pcl_path)
training_generator = torch.utils.data.DataLoader(training_set, **params)

densenet = DenseNet(input_size, upsampling_factor)
densenet = densenet.to('cuda')
mse = nn.MSELoss()

optim = torch.optim.Adam(densenet.parameters(), lr=1e-4)

counter = 0

points_loss_val = 0.001
pbar = tqdm(training_generator)
counter_ep = 1
counter = 0
for i in range(8):
    print(f"{counter_ep} epoch")
    for data, target in pbar:
        counter += 1
        optim.zero_grad()
        data = data.to('cuda')
        target = target.to('cuda')
        out = densenet(data)
        out = out.reshape(-1, 2048, 3)

        target = target.view(-1, 2048, 3)
        #print(out)
        loss = kaolin.metrics.pointcloud.chamfer_distance(out, target)  # Используем второе облако точек для расчета потерь
        loss = torch.mean(loss) + points_loss_val * point_loss(out)
        pbar.set_description(f"Loss: {loss.item()}")
        if counter % 250 == 0:
            print(counter, loss)
        loss.backward()
        optim.step()
    counter_ep += 1
    torch.save(densenet.state_dict(), f'densenet_{counter_ep}_epoch.pt')
print("end of learning")
torch.save(densenet.state_dict(), 'densenet.pt')

  0%|          | 0/1095 [00:00<?, ?it/s]

1 epoch
250 tensor(0.0167, device='cuda:0', grad_fn=<AddBackward0>)
500 tensor(0.0137, device='cuda:0', grad_fn=<AddBackward0>)
750 tensor(0.0122, device='cuda:0', grad_fn=<AddBackward0>)
1000 tensor(0.0140, device='cuda:0', grad_fn=<AddBackward0>)
2 epoch
1250 tensor(0.0124, device='cuda:0', grad_fn=<AddBackward0>)
1500 tensor(0.0129, device='cuda:0', grad_fn=<AddBackward0>)
1750 tensor(0.0137, device='cuda:0', grad_fn=<AddBackward0>)
2000 tensor(0.0111, device='cuda:0', grad_fn=<AddBackward0>)
3 epoch
2250 tensor(0.0125, device='cuda:0', grad_fn=<AddBackward0>)
2500 tensor(0.0102, device='cuda:0', grad_fn=<AddBackward0>)
2750 tensor(0.0116, device='cuda:0', grad_fn=<AddBackward0>)
3000 tensor(0.0128, device='cuda:0', grad_fn=<AddBackward0>)
3250 tensor(0.0130, device='cuda:0', grad_fn=<AddBackward0>)
4 epoch
3500 tensor(0.0114, device='cuda:0', grad_fn=<AddBackward0>)
3750 tensor(0.0112, device='cuda:0', grad_fn=<AddBackward0>)
4000 tensor(0.0131, device='cuda:0', grad_fn=<AddBackwar

In [ ]:
tens = torch.randn((1, 3292, 3)).to('cuda')
densenet.predict(tens, 3).shape

In [64]:
def val(model, valid_set, num_examples = 100):
    model = model.to('cuda')
    elems = np.random.choice(len(valid_set), num_examples)
    metric = torch.tensor([0.0])
    metric = metric.to('cuda')

    for i in elems:
        pcl1, pcl2 = valid_set[i]
        pcl1 = pcl1.to('cuda').unsqueeze(0)
        pcl2 = pcl2.to('cuda').unsqueeze(0)
        pcl1 = pcl1.view(-1, 1024, 3)
        pcl2 = pcl2.view(-1, 2048, 3)
        out = model(pcl1)
        out = out.reshape(-1, 2048, 3)
        metric += kaolin.metrics.pointcloud.f_score(out, pcl2) / num_examples

    return metric

In [65]:
upsampling_factor = 2
input_size = 3
densenet = DenseNet(input_size, upsampling_factor)
densenet.load_state_dict(torch.load('densenet.pt'))
pcl_path = '/content/extracted_files/ShapeNet_pointclouds'
json_path = '/content/extracted_files/splits/val_models.json'
valid_set = BatchGen_DenceNet(json_path, pcl_path)

score = val(densenet, valid_set, num_examples = 10)
